In [3]:
import pandas as pd
import numpy
import spacy
%matplotlib inline

In [4]:
reviewsdf = pd.read_csv('deceptive-opinion.csv')

In [5]:
reviewsdf.head(5)

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [6]:
reviewsdf.tail(5)

,deceptive,hotel,polarity,source,text
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i..."
1599,deceptive,amalfi,negative,MTurk,"As a former Chicagoan, I'm appalled at the Ama..."


In [7]:
#visualizations go here

In [8]:
reviewsdf.pivot_table(
    columns=['deceptive','polarity'],
    values=['deceptive','polarity'],
    index = ['hotel'],
    aggfunc = len).fillna(0)

source                                 text           \
deceptive        deceptive          truthful          deceptive            
polarity          negative positive negative positive  negative positive   
hotel                                                                      
affinia                 20       20       20       20        20       20   
allegro                 20       20       20       20        20       20   
amalfi                  20       20       20       20        20       20   
ambassador              20       20       20       20        20       20   
conrad                  20       20       20       20        20       20   
fairmont                20       20       20       20        20       20   
hardrock                20       20       20       20        20       20   
hilton                  20       20       20       20        20       20   
homewood                20       20       20       20        20       20   
hyatt                   20       20       20       20        20       20   
intercontinental        20       20       20       20        20       20   
james                   20       20       20       20        20       20   
knickerbocker           20       20       20       20        20       20   
monaco                  20       20       20       20        20       20   
omni                    20       20       20       20        20       20   
palmer                  20       20       20       20        20       20   
sheraton                20       20       20       20        20       20   
sofitel                 20       20       20       20        20       20   
swissotel               20       20       20       20        20       20   
talbott                 20       20       20       20        20       20   

                                    
deceptive        truthful           
polarity         negative positive  
hotel                               
affinia                20       20  
allegro                20       20  
amalfi                 20       20  
ambassador             20       20  
conrad                 20       20  
fairmont               20       20  
hardrock               20       20  
hilton                 20       20  
homewood               20       20  
hyatt                  20       20  
intercontinental       20       20  
james                  20       20  
knickerbocker          20       20  
monaco                 20       20  
omni                   20       20  
palmer                 20       20  
sheraton               20       20  
sofitel                20       20  
swissotel              20       20  
talbott                20       20

In [9]:
nlpcopy=reviewsdf.copy()
nlpcopy.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [10]:
import nltk

In [11]:
from nltk.corpus import stopwords
import re

In [12]:
stopwds=set(stopwords.words('english'))

In [13]:
import en_core_web_md

In [14]:
nlp=en_core_web_md.load()

In [15]:
#parsed = nlp.pipe(iter(data_nlp['text']), batch_size=1, n_threads=4)

from sklearn.feature_extraction.text import TfidfVectorizer
 
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))

 
# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(nlpcopy['text'])

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, nlpcopy['deceptive'], test_size=0.1, random_state=42)


In [17]:
from sklearn.svm import LinearSVC
 
# initialise the SVM classifier
clf = LinearSVC()
 # train the classifier
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [18]:
preds = clf.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

['deceptive', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive']
526     deceptive
354      truthful
168      truthful
135      truthful
937      truthful
1544    deceptive
1253    deceptive
237      truthful
478     deceptive
650     deceptive
Name: deceptive, dtype: object


In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

0.89375


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

             precision    recall  f1-score   support

  deceptive       0.86      0.94      0.89        77
   truthful       0.93      0.86      0.89        83

avg / total       0.90      0.89      0.89       160



In [21]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[72  5]
 [12 71]]


http://www.developintelligence.com/blog/2017/03/predicting-yelp-star-ratings-review-text-python/
texts = ["I really hated my stay at The NotARealName Hotel", "Had a really really great stay at NotARealName - would recommend to everyone"]
 
#### note that we only call .transform() here and not .fit_transform()
#### as we want to keep the vocabulary from the previous experiments
vecs = vectorizer.transform(texts)
 
#### predict a positive or negative label for each input
print(classifier.predict(vecs))
 
>>>['n' 'p']

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [23]:
# initialise the SVM classifier
clf = LogisticRegression()
#LinearDiscriminantAnalysis()
#GaussianNB()))
#SVC()
 # train the classifier
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
#

In [25]:
preds = clf.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

['deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive']
526     deceptive
354      truthful
168      truthful
135      truthful
937      truthful
1544    deceptive
1253    deceptive
237      truthful
478     deceptive
650     deceptive
Name: deceptive, dtype: object


In [26]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

0.875


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

             precision    recall  f1-score   support

  deceptive       0.83      0.94      0.88        77
   truthful       0.93      0.82      0.87        83

avg / total       0.88      0.88      0.87       160



In [28]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[72  5]
 [15 68]]


###initialise the SVM classifier
clf = GaussianNB()
#clf = LinearDiscriminantAnalysis()
#LinearDiscriminantAnalysis()
#GaussianNB()
#SVC()
 #train the classifier
clf.fit(X_train, y_train)
#doesn't work on a sparse matrix

In [29]:
#Now import the yelp hotel reviews

In [30]:
import json

businesses_filepath = 'yelp_training_set_business.json'

hotel_ids = set()

# open the businesses file
#f = open(fname, encoding="utf-8")
with open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a hotel, skip to the next one
        if u'Hotels' not in business[u'categories']:
            continue
            
        # add the hotel business id to our hotel_ids set
        hotel_ids.add(business[u'business_id'])


hotel_ids = frozenset(hotel_ids)

# print the number of unique hotel ids in the dataset
print ('{:,}'.format(len(hotel_ids)), u'hotels in the dataset.')

284 hotels in the dataset.


In [31]:
review_txt_filepath = 'review_text_all.txt'

In [32]:
review_json_filepath = 'yelp_training_set_review.json'
# 
if 0 == 0:
    
    review_count = 0

    # create & open a new file in write mode
    with open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a hotel, skip to the next one
                if review[u'business_id'] not in hotel_ids:
                    continue

                # write the hotel review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print (u'''Text from {:,} hotel reviews written to the new txt file.'''.format(review_count))
    
else:
    
    with open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print (u'Text from {:,} hotel reviews in the txt file.'.format(review_count + 1))

Text from 5,034 hotel reviews written to the new txt file.


In [36]:
revdf=pd.read_json('yelp_training_set_review.json')
revdf
# load json as string
#json.load((open('yelp_training_set_business.json')))

ValueError: Trailing data

In [35]:
with open('yelp_training_set_business.json', 'r') as content_file:
    content = content_file.read()
records = json.loads(content)

JSONDecodeError: Extra data: line 2 column 1 (char 414)